<a href="https://colab.research.google.com/github/du6293/DL_study/blob/main/4%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 교차 엔트로피 오차

In [ ]:
import numpy as np

In [ ]:
# 오차제곱합
def sum_squares_error(y,t):
  return 0.5 * np.sum((y-t)**2)

In [ ]:
# 교차 엔트로피 오차
def cross_entropy_error(y,t):  # y와 t는 넘파이 배열
  delta = 1e-7
  return -np.sum(t * np.log(y+ delta))

### MNIST 미니배치 학습 (데이터의 일부를 추려 전체의 근사치로 이용)

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, x_test),(x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)
print(x_train.shape) # (60000,784)
print(t_train.shape) # (60000,10)

In [ ]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) # train데이터에서 무작위로 10장 빼냄>>미니배치로 뽑아낼 인덱스로 활용

In [ ]:
#(배치용) 교차 엔트로피 오차 구현(정답 레이블 t가 one-hot-encoding일 때)
def cross_entropy_error(y,t):  # y: 신경망의 출력, t : 정답 레이블
  if y.ndim == 1:
    t = t.reshape(1,t.size)  # reshape : 데이터의 형상을 바꿈
    y = y.reshape(1,y.size)
  batch_size = y.shape[0]
  return -np.sum(t*np.log(y+1e-7)) / batch_size  # 배치의 크기로 나누어 정규화한 후 1장당 평균의 교차 엔트로피 오차를 계산

In [ ]:
#(배치용) 교차 엔트로피 오차 구현(정답 레이블 t가 숫자 레이블일 때)
def cross_entropy_error(y,t):  # y: 신경망의 출력, t : 정답 레이블
  if y.ndim == 1:
    t = t.reshape(1,t.size)  # reshape : 데이터의 형상을 바꿈
    y = y.reshape(1,y.size)
  batch_size = y.shape[0]
  return -np.sum(np.log(y[np.arrange(batch_size),t] + 1e-7)) / batch_size

### 수치 미분

In [ ]:
# 수치미분의 나쁜 구현 예  > 오차발생
def numerical_diff(f,x):
  h = 10e-50
  return (f(x+h)-f(x)) / h


In [ ]:
# 수치미분의 올바른 구현 예
def numerical_diff(f,x):
  h = 1e-4  # 0.0001
  return (f(x+h) - f(x-h)) / (2 * h)

In [ ]:
# y = 0.01x2 + 0.1x 구현
def function_1(x):
  return 0.01 * x ** 2 + 0.1 * x

In [ ]:
import numpy as np
import matplotlib.pylab as plt

x = np.arange(0.0, 20.0, 0.1) # 0에서 20까지 0.1 간격의 배열 x를 만든다.
y = function_1(x)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.plot(x,y)
plt.show()

### 편미분
변수가 여럿인 함수에 대한 미분

In [ ]:
def function_2(x):
  return x[0] ** 2 + x[1] ** 2
  # 또는 return np.sum(x**2)
  

### 기울기

In [ ]:
def numerical_gradient(f,x):
  h = 1e-4 # 0.0001
  grad = np.zeros_like(x) # x와 형상이 같고 원소가 모두 0인 배열을 생성

  for idx in range(x.size):
    tmp_val = x[idx]
    # f(x+h) 계산
    x[idx] = tmp_val + h
    fxh1 = f(x)

    # f(x-h) 계산
    x[idx] = tmp_val - h
    fxh2 = f(x)

    grad[idx] = (fxh1 - fxh2) / (2 * h)
    x[idx] = tmp_val  # 값 복원

  return grad

In [ ]:
print(numerical_gradient(function_2,np.array([3.0,4.0])))
print(numerical_gradient(function_2,np.array([0.0,2.0])))
print(numerical_gradient(function_2,np.array([3.0,0.0])))

### 경사 하강법

In [ ]:
def gradient_descent(f, init_x, lr = 0.01, step_num = 100):  # lr : 학습률, step_num = 반복횟수
  x = init_x

  for i in range(step_num):
    grad = numerical_gradient(f,x)  # 기울기를 구함
    x -= lr * grad  # 기울기 * 학습률
  return x

In [ ]:
# 경사법으로 f(x0,x1) = x02+x12의 최솟값을 구하기
def fucntion_2(x):
  return x[0] ** 2 + x[1] ** 2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x = init_x, lr = 0.1, step_num = 100)

In [ ]:
# 학습률이 너무 큰 경우 : lr = 10.0
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x = init_x, lr = 10.0, step_num = 100)




In [ ]:
# 학습률이 너무 작은 경우 : lr = 1e-10
init_x = np.array([-3.0,4.0])
gradient_descent(function_2, init_x = init_x, lr = 1e-10, step_num = 100)

### 신경망에서의 기울기


In [ ]:
# 기울기를 구하는 코드
import sys, os
sys.path.append("/content/drive/MyDrive/bottom/deep-learning-from-scratch-master/deep-learning-from-scratch-master/common")
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
  def __init__(self):
    self.w = np.random.randn(2,3)  # 정규분포로 초기화

  def predict(self, x):
    return np.dot(x,self.w)

  def loss(self, x, t):
    z = self.predict(x)
    y = softmax(z)
    loss = cross_entropy_error(y,t)

    return loss

In [ ]:
net = simpleNet()
print(net,w)  # 가중치 매개변수
x = np.array([0.6, 0.9])

p = net.predict(x)
print(p)

np.argmax(p)

t = np.array([0,0,1]) # 정답 레이블
net.loss(x,t)

### 2층 신경망 클래스 구현하기

In [ ]:
import sys, os 
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:

  def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
    # 가중치 초기화
    self.params = {}  # 신경망의 매개변수를 보관하는 딕셔너리 변수
    self.params['w1'] = weight_init_std * np.random.randn(input_size, hidden_size)  # 1번째 층의 가중치
    self.params['b1'] = np.zeros(hidden_size)                                       # 1번째 층의 편향
    self.params['w2'] = weight_init_std * np.random.randn(hidden_size, output_size) # 2번째 층의 가중치
    self.params['b2'] = np.zeros(output_size)                                       # 2번째 층의 편향

    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x,w1) + b1
    z1 = sigmoid(a1)
    z2 = np.dot(z1,w2) + b2
    y = softmax(z2)

    return y


    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
      y = self.predict(x)
      return cross_entropy_error(y,t)

    def accuracy(self, x, t):
      y = self.predict(x)
      y = np.argmax(y, axis = 1)
      y = np.argmax(t, axis = 1)

      accuracy = np.sum(y==t) / float(x.shape[0])
      return accuracy
      
      # x:입력 데이터, t: 정답 레이블
    def numerical_gradient(self,x,t):
      loss_w = lambda w: self.loss(x,t)

      grads = {}  # 기울기를 보관하는 딕셔너리 변수
      grads['w1'] = numerical_gradient(loss_w, self.params['w1'])  # 1번째 층의 가중치의 기울기
      grads['b1'] = numerical_gradient(loss_w, self.params['b1'])  # 1번째 층의 편향의 기울기
      grads['w3'] = numerical_gradient(loss_w, self.params['w2'])  # 2번재 층의 가중치의 기울기
      grads['b2'] = numerical_gradient(loss_w, self.params['b2'])  # 2번째 층의 편향의 기울기

      return grads

### 미니 배치 학습 구현하기


In [ ]:
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_encoding = True)

train_loss_list = []

# 하이퍼파라미터
iters_num = 10000 # 반복횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니 배치 크기
learning_rate = 0.1
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

for i in range(iters_num):
  # 미니배치 획득
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산 
  grad = network.numerical_gradient(x_batch, t_batch)
  # grad = network.gradient(x_batch, t_batch)  # 성능 개선판 !

  # 매개변수 갱신
  for key in ('w1','b1','w2','b2'):
    network.params[key] -= learning_rate * grad[key]

  # 학습 경과 기록
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)



In [ ]:
# 시험 데이터로 평가하기
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet
(x_train, t_train),(x_test,t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

# 하이퍼 파라미터
iters_num = 10000 # 반복 횟수를 적절히 설정
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list =[]

# 1에폭 당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
  # 미니배치 획득
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산
  grad = network.numerical_gradient(x_batch, t_batch)
  # grad = network.gradient(x_batch, t_batch) # 성능 개선판 !

  # 매개변수 갱신
  for key in ('w1','b1','w2','b2'):
    network.params[key] -= learning_rate * grad[key]

  # 학습 경과 기록
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  # 1에폭 당 정확도 계산
  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print("train acc, test acc | " + str(train_acc) + " , " + str(test_acc))